In [263]:
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
import numpy as np
import ast

In [264]:
# Dresden  Station ID = 1048 , Stations-Kennziffer 10488 (Klotzsche)
# Freiburg: Station ID - 1443, 10803  - ? 
 
# 10 minute data https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/

In [265]:
# Columns
# TT_10 temperature
# RF_10 relative humidity 
# RWS_10 Sum of the precipitation height of the previous 10 minutes
# FF_10 mean wind speed during the previous 10 minutes



# Dresden

In [266]:
df_air_DD = pd.read_csv("../data/weather/Dresden/Dresden temperature.txt", sep=";")

In [ ]:
df_air_DD.head()

In [ ]:
df_air_DD.describe()

In [269]:
df_precip_DD = pd.read_csv("../data/weather/Dresden/Dresden precipitation.txt", sep=";")

In [ ]:
df_precip_DD.head()

In [ ]:
df_precip_DD.describe()

In [272]:
df_wind_DD = pd.read_csv("../data/weather/Dresden/Dresden wind.txt", sep=";")

In [ ]:
df_wind_DD.head()

In [ ]:
for df in [df_air_DD, df_precip_DD, df_wind_DD]:
    df.columns = [col.strip() for col in df.columns]
    df["MESS_DATUM"] = pd.to_datetime(df["MESS_DATUM"], format= "%Y%m%d%H%M")
    df.drop(columns=["QN", "eor"], inplace=True, errors="ignore")
    print(len(df))

In [ ]:
df_air_DD

In [276]:
df_tmp = pd.merge(df_air_DD, df_precip_DD, on=["MESS_DATUM", "STATIONS_ID"])

In [277]:
df_DD = pd.merge(df_tmp, df_wind_DD, on=["MESS_DATUM", "STATIONS_ID"])

In [ ]:
len(df_DD)

In [279]:
assert len(df_DD.STATIONS_ID.unique().tolist())==1

In [ ]:
df_DD["MESS_DATUM"].min()

In [ ]:
df_DD["MESS_DATUM"].max()

In [ ]:
pd.to_datetime("2024-01-01")

In [ ]:
DD_dt_flt = ((df_DD["MESS_DATUM"] >= "2024-01-01") & (df_DD["MESS_DATUM"] < "2024-04-01")) | \
 ((df_DD["MESS_DATUM"] >= "2024-09-01") & (df_DD["MESS_DATUM"] < "2024-11-01")) 
len(df_DD[DD_dt_flt])

In [ ]:
len(df_DD)

In [285]:
df_DD = df_DD[DD_dt_flt]

In [ ]:
df_DD

In [287]:
df_DD = df_DD[["MESS_DATUM", "TT_10", "RF_10", "RWS_10", "FF_10"]]

In [288]:
#df_DD['city']="Dresden"

In [ ]:
df_DD.head()

# Freiburg

In [290]:
df_air_recent_FB = pd.read_csv("../data/weather/Freiburg/Freiburg recent temperature.txt", sep=";")

In [ ]:
df_air_recent_FB.head()

In [292]:
df_air_hist_FB = pd.read_csv("../data/weather/Freiburg/Freiburg historical air temperature.txt", sep=";")

In [293]:
df_air_FB = pd.concat([df_air_recent_FB, df_air_hist_FB])

In [294]:
df_air_FB = df_air_FB.drop_duplicates()

In [ ]:
df_air_FB.head()

In [296]:
df_precip_FB_recent = pd.read_csv("../data/weather/Freiburg/Freiburg recent precipitation.txt", sep=";")

In [297]:
df_precip_FB_hist = pd.read_csv("../data/weather/Freiburg/Freiburg historical precipitation.txt", sep=";")

In [298]:
df_precip_FB = pd.concat([df_precip_FB_recent, df_precip_FB_hist])

In [ ]:
len(df_precip_FB)

In [300]:
df_precip_FB = df_precip_FB.drop_duplicates()

In [301]:
df_wind_FB_recent = pd.read_csv("../data/weather/Freiburg/Freiburg recent wind.txt", sep=";")

In [302]:
df_wind_FB_hist = pd.read_csv("../data/weather/Freiburg/Freiburg historical wind.txt", sep=";")

In [303]:
df_wind_FB = pd.concat([df_wind_FB_recent, df_wind_FB_hist])

In [304]:
df_wind_FB = df_wind_FB.drop_duplicates()

In [ ]:
for df in [df_air_FB, df_precip_FB, df_wind_FB]:
    df.columns = [col.strip() for col in df.columns]
    df["MESS_DATUM"] = pd.to_datetime(df["MESS_DATUM"], format= "%Y%m%d%H%M")
    df.drop(columns=["QN", "eor"], inplace=True, errors="ignore")
    print(len(df))

In [306]:
df_tmp = pd.merge(df_air_FB, df_precip_FB, on=["MESS_DATUM", "STATIONS_ID"])

In [307]:
df_FB = pd.merge(df_tmp, df_wind_FB, on=["MESS_DATUM", "STATIONS_ID"])

In [ ]:
df_FB.head()

In [309]:
assert len(df_FB.STATIONS_ID.unique().tolist())==1

In [ ]:
df_FB["MESS_DATUM"].min()

In [ ]:
df_FB["MESS_DATUM"].max()

In [ ]:
FB_dt_flt = ((df_FB["MESS_DATUM"] >= "2023-06-01") & (df_FB["MESS_DATUM"] < "2023-08-01")) | \
 ((df_FB["MESS_DATUM"] >= "2024-09-01") & (df_FB["MESS_DATUM"] < "2024-11-01")) 
len(df_FB[FB_dt_flt])

In [ ]:
len(df_FB)

In [314]:
df_FB = df_FB[FB_dt_flt]

In [ ]:
df_FB

In [316]:
df_FB = df_FB[["MESS_DATUM", "TT_10", "RF_10", "RWS_10", "FF_10"]]

In [317]:
df_FB = df_FB.replace(-999, np.nan)

In [318]:
# df_FB['city']="Freiburg"

# unite

In [ ]:
df_FB.head()

In [ ]:
df_DD.head()

In [321]:
col_renaming = { "MESS_DATUM": "Date", "TT_10": "Temperature", "RF_10": "Humidity", "RWS_10": "Precipitation", "FF_10": "Wind" }

In [ ]:
df_DD.rename(columns=col_renaming, inplace=True)

In [ ]:
df_FB.rename(columns=col_renaming, inplace=True)

In [324]:
df_DD.set_index("Date", inplace=True)

In [325]:
df_FB.set_index("Date", inplace=True)

In [ ]:
len(df_DD)

In [ ]:
21888/14640

In [328]:
grouped_DD = df_DD.resample("30min").mean()

In [329]:
grouped_FB = df_FB.resample("30min").mean()

In [ ]:
FB_dt_flt = ((grouped_FB.index >= "2023-06-01") & (grouped_FB.index < "2023-08-01")) | \
 ((grouped_FB.index >= "2024-09-01") & (grouped_FB.index < "2024-11-01")) 
len(grouped_FB[FB_dt_flt])

In [331]:
grouped_FB = grouped_FB[FB_dt_flt]

In [ ]:
grouped_FB.describe()

In [ ]:
grouped_FB[grouped_FB.Precipitation == -999]

In [334]:
assert len(df_FB)/len(grouped_FB)==3

In [ ]:
grouped_FB.tail()

In [ ]:
len(df_DD.resample("30min").mean())

In [ ]:
21888/3

In [ ]:
df_DD.head()

In [ ]:
df_DD.resample("30min").mean()

In [340]:
DD_dt_flt = ((grouped_DD.index >= "2024-01-01") ) & (grouped_DD.index < "2024-04-01") | \
 ((grouped_DD.index >= "2024-09-01") & (grouped_DD.index < "2024-11-01")) 
len(grouped_DD[DD_dt_flt])
grouped_DD = grouped_DD[DD_dt_flt]

In [ ]:
df_DD

In [ ]:
grouped_DD.describe()

In [ ]:
grouped_FB.describe()

In [ ]:
grouped_FB.info()

In [346]:
df_missing_FB = grouped_FB[grouped_FB.Temperature.isna()].copy()

In [ ]:
df_missing_FB

In [348]:
for col in ["Temperature", "Humidity", "Precipitation", "Wind"]:
    grouped_FB[col] = grouped_FB[col].interpolate(method = "nearest")

In [ ]:
grouped_FB.loc[df_missing_FB.index]

In [ ]:
len(df_FB)/3

In [ ]:
grouped_FB.describe()

In [353]:
assert len(df_DD)/len(grouped_DD)==3

In [354]:
time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
grouped_DD.to_pickle(f"../data/weather/df_Dresden_weather {time}.pkl")
grouped_DD.to_csv(f"../data/weather/df_Dresden_weather {time}.csv")

grouped_FB.to_pickle(f"../data/weather/df_Freiburg_weather {time}.pkl")
grouped_FB.to_csv(f"../data/weather/df_Freiburg_weather {time}.csv")

